In [1]:
import re
import qiskit
import numpy as np
from typing import List
from qiskit import *


In [2]:
from circuit_generator import generate_opt_circuit, construct_qcc_circuit
from fast_generator import fc_tree_commute_recur_lookahead_fast, update_observables, simulate_expectationval
from utilities import compare_lists


In [3]:
# test_observables = ['ZZZZZZZZ', 'XXXXYYYY', 'ZYZIXYZI', 'XYZXYZII']
# test_paulis = ['XXIIIIXY', 'IIXXXYII', 'IXXIXIIY', 'XIIXIXYI',
#                   'XXIIXYII', 'IXIXIXIY', 'XIXIXIYI', 'IIXXIIXY']
# test_params = [1.16692654, 0.27223177, -0.93402707, -0.92067998, 0.06852241, -0.42444632, -0.41270851, -0.01068001]

In [4]:
#simplified two strings:
test_observables = ['XXXXXX', 'YYYYYY', 'XYXYXY', 'YXYXYX', 'YYYXXX', 'XXXYYY', 'ZZZZZZ', 'ZZIIII', 'IIZZII', 'IIIIZZ', 'XXXXXZ','XXXXZZ', 'XXXZZZ','XXZZZZ','XZZZZZ','ZZZXXX']
test_paulis = ['XXXXXY', 'XXXIYI', 'IXIXXY', 'IXIIYI', 'IXXIXY', 'XXIXYI', 'IIIXIY', 'XIYIII']
test_params = [0.0944527, 0.04799566, -0.0590973, -0.05908328, 0.04114604, 0.02695483, 0.02604318, 0.03485649]

In [5]:
from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers

In [6]:
#first generate the original hamiltonian simulation circuit
origin_qc = construct_qcc_circuit(entanglers = test_paulis, params = test_params, barrier=False)
origin_qc.count_ops()['cx']

40

In [7]:
#generate the qiskit optimized circuit
origin_qiskit = transpile(origin_qc, optimization_level = 3, basis_gates = ["cx", "sx", "x", "rz"])
origin_qiskit.count_ops()['cx']

40

In [8]:
#simulate the circuit and measure the expectation values
orign_expect_vals = []
for obs in test_observables:
    expectation_val = simulate_expectationval(origin_qc, observable=obs, shots=100000)
    orign_expect_vals.append(expectation_val)

XXXXXX
YYYYYY
YXYXYX
XYXYXY
XXXYYY
YYYXXX
ZZZZZZ
ZZZZZZ
ZZZZZZ
ZZZZZZ
ZXXXXX
ZZXXXX
ZZZXXX
ZZZZXX
ZZZZZX
XXXZZZ


In [9]:
print(orign_expect_vals)

[0.09236, -0.0952, 0.004, -0.00248, 0.00554, 0.00378, 1.0, 0.99494, 0.9948, 0.9964, 0.00258, -0.0015, 0.00202, 0.00324, -0.00714, -0.00454]


In [10]:
#Then we run QuCLEAR to optimize the circuit
opt_qc, append_clifford, sorted_entanglers = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False)


In [11]:
#QuCLEAR generates the optimized circuit with reduced number of CNOT gates
opt_qc.count_ops()['cx']

17

In [12]:
append_clifford.count_ops()

OrderedDict([('cx', 20), ('h', 18), ('s', 9), ('x', 4), ('swap', 2), ('y', 1)])

In [13]:
append_clifford.draw()

┌───┐┌───┐                  ┌───┐┌───┐     ┌───┐┌───┐     »
q_0: ────────────■──┤ H ├┤ H ├──────X───────────┤ X ├┤ H ├──■──┤ H ├┤ S ├─────»
               ┌─┴─┐├───┤├───┤┌───┐ │      ┌───┐└─┬─┘└───┘  │  └───┘├───┤┌───┐»
q_1: ───────■──┤ X ├┤ H ├┤ S ├┤ H ├─┼──────┤ X ├──┼─────────┼────■──┤ S ├┤ H ├»
     ┌───┐  │  └───┘└───┘└───┘└───┘ │ ┌───┐└─┬─┘  │       ┌─┴─┐┌─┴─┐└───┘├───┤»
q_2: ┤ H ├──┼───────────────────────X─┤ X ├──┼────■───────┤ X ├┤ X ├──■──┤ X ├»
     └───┘┌─┴─┐┌───┐┌───┐             └─┬─┘  │            └───┘└───┘┌─┴─┐├───┤»
q_3: ──■──┤ X ├┤ S ├┤ H ├───────────────┼────■──────────────────────┤ X ├┤ S ├»
       │  ├───┤└───┘└───┘               │                           └───┘└───┘»
q_4: ──┼──┤ H ├─────────────────────────┼─────────────────────────────────────»
     ┌─┴─┐├───┤┌───┐┌───┐               │                                     »
q_5: ┤ X ├┤ H ├┤ S ├┤ H ├───────────────■─────────────────────────────────────»
     └───┘└───┘└───┘└───┘                                                     »
«     ┌───┐                    ┌───┐                            ┌───┐┌───┐     »
«q_0: ┤ X ├───────■────────────┤ X ├───────■─────────────X──────┤ X ├┤ H ├──■──»
«     └─┬─┘       │  ┌───┐┌───┐└─┬─┘     ┌─┴─┐     ┌───┐ │      └─┬─┘└───┘  │  »
«q_1: ──■─────────┼──┤ X ├┤ H ├──┼────■──┤ X ├──■──┤ X ├─┼────────┼─────────┼──»
«                 │  └─┬─┘└───┘  │    │  └───┘  │  └───┘ │        │         │  »
«q_2: ────────────┼────┼─────────┼────┼─────────┼────────┼────────┼─────────┼──»
«     ┌───┐┌───┐┌─┴─┐  │  ┌───┐  │    │         │        │        │         │  »
«q_3: ┤ H ├┤ X ├┤ X ├──■──┤ Y ├──┼────┼─────────┼────────┼────────┼─────────┼──»
«     └───┘└─┬─┘└───┘     └───┘  │  ┌─┴─┐       │        │ ┌───┐  │  ┌───┐  │  »
«q_4: ───────┼───────────────────┼──┤ X ├───────┼────────┼─┤ X ├──┼──┤ X ├──┼──»
«            │  ┌───┐┌───┐       │  └───┘     ┌─┴─┐┌───┐ │ └─┬─┘  │  └───┘┌─┴─┐»
«q_5: ───────■──┤ S ├┤ H ├───────■────────────┤ X ├┤ H ├─X───■────■───────┤ X ├»
«               └───┘└───┘                    └───┘└───┘                  └───┘»
«     ┌───┐┌───┐┌───┐
«q_0: ┤ S ├┤ H ├┤ S ├
«     └───┘└───┘└───┘
«q_1: ───────────────
«                    
«q_2: ───────────────
«                    
«q_3: ───────────────
«                    
«q_4: ───────────────
«     ┌───┐          
«q_5: ┤ X ├──────────
«     └───┘

In [14]:
# based on the observables and the appended clifford circuit, we can calculate the updated observables
updated_signs, updated_observables = update_observables(test_observables, [append_clifford])
print(updated_observables)

['XIIYZZ', 'XXIYZI', 'XXZXZI', 'XIZXZZ', 'XXZXZZ', 'XIZXZI', 'IXIIIZ', 'ZXXXII', 'XIYXXI', 'YIZIXZ', 'XIIYZY', 'XZYYYY', 'ZZXYIZ', 'IZIYYY', 'XXIXXY', 'YYXIZZ']


In [15]:
#Run the same circuit simulation process for the updated observables
opt_expect_vals = []
for idx, obs in enumerate(updated_observables):
    expectation_val = simulate_expectationval(opt_qc, observable=obs, shots=100000)
    if updated_signs[idx] == '+1':
        updated_sign = 1
    elif updated_signs[idx] == '-1':
        updated_sign = -1
    else:
        raise Exception("incorrect sign")
    opt_expect_vals.append(updated_sign * expectation_val)

ZZYZZX
ZZYZXX


ZZXZXX
ZZXZZX
ZZXZXX
ZZXZZX
ZZZZXZ
ZZXXXZ
ZXXYZX
ZXZZZY
YZYZZX
YYYYZX
ZZYXZZ
YYYZZZ
YXXZXX
ZZZXYY


In [16]:
opt_expect_vals

[0.09224,
 -0.09296,
 0.00274,
 0.00122,
 0.00286,
 -0.0002,
 1.0,
 0.99556,
 0.99534,
 0.99642,
 0.0025,
 -0.00154,
 0.00702,
 0.00028,
 -0.00018,
 -0.00134]

In [17]:
orign_expect_vals

[0.09236,
 -0.0952,
 0.004,
 -0.00248,
 0.00554,
 0.00378,
 1.0,
 0.99494,
 0.9948,
 0.9964,
 0.00258,
 -0.0015,
 0.00202,
 0.00324,
 -0.00714,
 -0.00454]

Validate the results

In [18]:
try:
    compare_lists(orign_expect_vals, opt_expect_vals, tolerance = 0.01)
    print("Lists differences are within acceptable limits.")
except ValueError as e:
    print(e)

Lists differences are within acceptable limits.
